# Opertura d'una nova oficina i predicció de potencials localitzacions futures
 

|   INTEGRANTS                    | Arnau Muñoz Barrera     | Míriam López Sánchez       | Luis Martínez Zamora       | Marc Rodríguez Vitolo
| -------------------------- | ----------------------- | -------------------------- | -------------------------- | --------------------------

### 1. Objectius

L'objectiu d'aquest projecte es centra en buscar ubicacions idònies per obrir noves oficines de la Caixa d'Enginyers, valorant tant la possiblitat d'oficines fixes com oficinse mòbils que arriben a una determinada zona. A més, es desenvoluparà també un model predictiu amb l'ajuda d'IA que permeti identificar potencials localitzacions futures.

### 2. Font de les dades

Com a dades per analitzar en aquest projecte s'han agafat conjunts de dades extrets de fonts oficials: INE, BdE.

### 3. Metodologia seguida

- Recerca de dades
- Anàlisi de dades amb python (realització de gràfiques i extracció d'estadístiques)
- Interpretació de resultats
- Elecció de les millors ubicacions posibles
- Entrenament del model predictiu
- Test del model predictiu
- Extracció de conclusions


### 4. Anàlisi de les dades


